In [1]:
"""
This script is used to generate all possible combination of Options 
and store the results in separate Excel files.
Files are named by the combination of `OPT` inside `Options` sheet.
For example, in part 1
"1_1_1_1_1_1_1_1_1_1_1.xlsx" represents
['Massivbauweise', 'Mit UG', 'Beheizt', 'Mit abgehängte Installationsdecke', 'Mit abgehängte Installationsdecke', 'Bodenbelag mit Parkett', 'Bodenbelag mit Parkett', 'Mit Dämmung', 'Betonwand', 'Flachdach', 'Ja']

Author: Gul Sena Altintas
Modified from Ao Chen. Place: Zweistein.
"""

'\nThis script is used to generate all possible combination of Options \nand store the results in separate Excel files.\nFiles are named by the combination of `OPT` inside `Options` sheet.\nFor example, \n"1_1_1_1_1_1_1_1_1_1_1.xlsx" represents\n[\'Massivbauweise\', \'Mit UG\', \'Beheizt\', \'Mit abgehängte Installationsdecke\', \'Mit abgehängte Installationsdecke\', \'Bodenbelag mit Parkett\', \'Bodenbelag mit Parkett\', \'Mit Dämmung\', \'Betonwand\', \'Flachdach\', \'Ja\']\n\nAuthor: Gul Sena Altintas\nModified from Ao Chen. Place: Zweistein.\n'

In [1]:
# uncomment to install missing packages
# ! pip install xlwings pandas openpyxl numpy

In [3]:
import xlwings as xw
from openpyxl import Workbook, load_workbook
import pandas as pd
from datetime import date
from pandas import ExcelWriter
import re
import itertools
import numpy as np

Currently, processing of two parts are implemented, if in the future more parts are added define the following variables:

    part_no = x
    opt_range_x: (start_row - 1, end_row - 1, start_col -1, end_column - 1)
        where this is defined in the Options sheet
    opt_ranges = [..., opt_range_x]
    target_range_x = excel range of the calculated target table
    target_ranges = [..., target_range_x]
    
    -1 because we need the python indices
    
Excel format:
- Conditions must be defined as `na` for no condition, `ONLY IF CONDITIONED_CELL = CONDITIONED_VAL -->`.
- For each cell, `OPT` must start from 1 and be incremented.

In [4]:
excel_name = '031122_OptiBat_v3.xlsx' # TODO: change excel name
part_no = 1 # Change based on which part you want to run

opt_range_1 = (2, 24, 2, 6)
opt_range_2 = (27, 67, 2, 6)
opt_ranges = [opt_range_1, opt_range_2]

target_range_1 = ('M4:AL16')
target_range_2 = ('M25:AL37')
target_ranges = [target_range_1, target_range_2]

optbat_db = pd.ExcelFile(excel_name)

Today = date.today().strftime("%d_%b_%Y")
results_path = f'{Today}_part{part_no}_results.xlsx' # TODO: change results name as needed

In [5]:
part = opt_ranges[part_no - 1]
target_range = target_ranges[part_no - 1]
    
Sheet_Options_df = optbat_db.parse('Options', header=None)
Sheet_Options_df = Sheet_Options_df.loc[part[0]: part[1], part[2]: part[3]]
Sheet_Options_df.columns = list(Sheet_Options_df.loc[part[0], :])
Sheet_Options_df = Sheet_Options_df.loc[part[0]+1:, :]
Sheet_Options_df = Sheet_Options_df.reset_index(drop=True)

In [6]:
Sheet_Options_df

,Condition,Cell,OPT,Options
0,na,G4,1,Massivbauweise
1,na,G4,2,Leichtbauweise
2,na,G4,3,Mischbauweise
3,na,G5,1,Mit UG
4,na,G5,2,Ohne UG
5,ONLY IF G5 = Mit UG -->,G6,1,Beheizt
6,ONLY IF G5 = Mit UG -->,G6,2,Unbeheizt
7,na,G7,1,Mit abgehängte Installationsdecke
8,na,G7,2,Mit abgehängte Installationsdecke-Mischbau
9,ONLY IF G5 = Mit UG -->,G8,1,Mit abgehängte Installationsdecke


In [8]:
Options_list = list(Sheet_Options_df['Options'])
Options_position_list = [x.strip() for x in list(Sheet_Options_df['Cell'])]
Options_condition_list = [x.strip() for x in list(Sheet_Options_df['Condition'])]
Options_opt_list = list(Sheet_Options_df['OPT'])

conditioned_on = ['na' if type(i) != str or not i.startswith('ONLY IF') else re.split(r'ONLY IF | =', i)[1] for i in Options_condition_list ]
condition = ['na' if type(i) != str or not i.startswith('ONLY IF') else re.split(r'ONLY IF | = | --', i)[2] for i in Options_condition_list ]


In [9]:
arrays = [
    Options_position_list,
    Options_opt_list
]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=["Cell", "OPT"])

In [10]:
Options_df = pd.DataFrame({'Options': Options_list, 'Conditioned_on': conditioned_on, 
                           'Condition': condition,}, index=index)
condition_codes = ['na' if x == 'na' else 
                   Options_df.loc[x][Options_df.loc[x, 'Options'] == cond].index[0] 
                   for (x, cond) in zip(conditioned_on, condition)]
Options_df['Condition_Code'] = condition_codes

In [11]:
def get_groups(options:pd.DataFrame, include_keys):
    tmp = options.groupby(level=0, sort=False).count()['Options']
    groups = [[(ind, x) for x in range(1, i+1)] for (ind, i) in zip(tmp.index, tmp) if ind in include_keys]
    return groups

tmp = Options_df.groupby(level=0, sort=False).count()['Options']
groups = [[(ind, x) for x in range(1, i+1)] for (ind, i) in zip(tmp.index, tmp)]


In [33]:
# linux
# wb = load_workbook(excel_name)
# sheet_Gebäude = wb['Gebäude']

# windows
wb = xw.Book(excel_name)
sheet_Gebäude = wb.sheets['Gebäude']


XlwingsError: Your platform only supports the instantiation via xw.Book(json=...)

In [ ]:
def process(ws, cells: tuple, options: pd.DataFrame):
    ''' 
    cells: eg. 
    (('G4', 1), ('G5', 1), ('G6', 1), ('G7', 1), ('G8', 1), 
    ('G9', 1), ('G10', 1), ('G11', 1), ('G12', 1), ('G13', 1), 
    ('G14', 2))
    
    Warning: Note that the function as it is may result in faulty behavior
    if any cell has a dependency for a cell below it (eg. G33 depends on G34)
    
    returns:
        target_name (str): sheet_name in the form 1_2_1_.. eg where 1, 2's correspond to opt
        description (str): the options concatenated by ,
        df (pd.DataFrame): calculated values from the original sheet, target range is chosen in 
            the beginning of the notebook
        
     '''
    targets = []
    description = ''
    # for each cell, we check if the conditions are met
    for cell in cells:
        cond_cell = options.loc[cell, 'Conditioned_on']
        # if the cell depends on another cell, check if the value of the dependent cell 
        # is equal to the value that allows this specific value
        # if the condition is not met, this cell is skipped, its results are not written to
        # target_name (the name of the sheet) or the description
        if cond_cell != 'na':
            cond = options.loc[cell, 'Condition']
            if ws.range(cond_cell).value != cond:
                continue
        # set cell value
        ws.range(cell[0]).value = options.loc[cell, 'Options']
        description += options.loc[cell, 'Options'] + ', '
        targets.append(cell[1])
    target_name = '_'.join([str(x) for x in targets])
    return target_name, description, pd.DataFrame(sheet_Gebäude.range(target_range).value)
                           

**Comment**: Please note that the following operation performs unnecessary computations (in the order of $\Pi group^{dependencies}$ hence it is significantly slow, as we actually don't need to check some cases, ie. any of G6, 7, 10, 11 when G5 is not 'Mit UG'. However, the other way around is prone to potential human mistakes and requires everything to be hard-coded.

In [13]:
all_combs = list(itertools.product(*groups))


In [ ]:
se = set()
with ExcelWriter(results_path) as writer:
    for comb in all_combs:
        target_name, description, df = process(sheet_Gebäude, comb, Options_df)
        df.loc[len(df), 0] = description
        print(description)
        se.add(target_name)
        df.to_excel(writer, sheet_name=target_name, header=None, index=None)
print(f'Written a total of {len(set)} sheets.')